In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
import matplotlib.pyplot as plt

importing NGSolve-6.2.2301


Consider the following. Find $u$ satisfying
\begin{equation}
-\nabla u - \omega^2 u = f \text{ in } \mathbb{R}^2,
\end{equation}
with $u$ satisfying the Sommerfeld radiation condition
\begin{equation}
\lim_{r\to\infty} r^{1/2} \left(\frac{\partial u}{\partial r} - i\omega u\right) = 0,
\end{equation}
where $r$ is the radial coordinate from the origin.

In [2]:
outer = Circle((0,0), 2).Face()
outer.edges.name = 'outerbnd'
inner = Circle((0,0), 1).Face()
inner.edges.name = 'innerbnd'
inner.faces.name ='inner'
pmlregion = outer - inner
pmlregion.faces.name = 'pmlregion'
geo = OCCGeometry(Glue([inner, pmlregion]), dim=2)

mesh = Mesh(geo.GenerateMesh (maxh=0.1))
mesh.Curve(3)


We set $f$ to be almost 0 everywhere except a small region away from the boundary,

In [3]:
f = exp(-20**2*((x-0.3)*(x-0.3)+y*y))
Draw(f, mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

Setting PML in pmlregion

In [4]:
mesh.SetPML(pml.Radial(rad=1,alpha=1j,origin=(0,0)), "pmlregion")

Solving H1 with $\omega=10$ using a direct inverse

In [5]:
fes = H1(mesh, order=4, complex=True)
u = fes.TrialFunction()
v = fes.TestFunction()

omega = 10

a = BilinearForm(fes)
a += grad(u)*grad(v)*dx - omega**2*u*v*dx
a += -1j*omega*u*v*ds("outerbnd")
a.Assemble()

b = LinearForm(f * v * dx).Assemble()

gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse() * b.vec
Draw(gfu);



WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

Trying again with my own implementation.

We want the field to decay exponentially to (near) 0 inside the PML.

\begin{equation}
A = \begin{cases}
1 & \text{ for } r \le r_{inner} \\
e^{\alpha r} & \text{ for } r > r_{inner} \\
0 & \text{ for } r = r_{outer}
\end{cases}
\end{equation}

In [18]:
outer = Circle((0,0), 2).Face()
outer.edges.name = 'outerbnd'
inner = Circle((0,0), 1).Face()
inner.edges.name = 'innerbnd'
inner.faces.name ='inner'
pmlregion = outer - inner
pmlregion.faces.name = 'pmlregion'
geo = OCCGeometry(Glue([inner, pmlregion]), dim=2)

mesh = Mesh(geo.GenerateMesh (maxh=0.1))
mesh.Curve(3)

geo = OCCGeometry(Glue([inner, pmlregion]), dim=2)

mesh = Mesh(geo.GenerateMesh (maxh=0.1))
mesh.Curve(3)

f = exp(-20**2*((x-0.3)*(x-0.3)+y*y))


print(mesh.GetMaterials())

fes = H1(mesh, order=4, complex=True, dirichlet='outerbnd')
u = fes.TrialFunction()
v = fes.TestFunction()
omega = 10

r = (x**2 + y**2)**(0.5)
r_inner = 1
alpha  = 10
amp_dict = {'inner': 1, 'pmlregion': exp(alpha * (r_inner - r))}
amp_dict2 = {'inner': 0, 'pmlregion': 1j * alpha * (r_inner - r)}

A = CoefficientFunction([amp_dict[mat] for mat in mesh.GetMaterials()])
A2 = CoefficientFunction([amp_dict2[mat] for mat in mesh.GetMaterials()])

Draw(A, mesh)

a = BilinearForm(fes)
a += SymbolicBFI(grad(u)*grad(v))
a += SymbolicBFI(-omega**2*u*v)
a += -1j*omega*u*v * ds('outerbnd')
a.Assemble()

x_prime = x + A2
y_prime = y + A2

f2 = exp(-20**2*((x_prime-0.3)*(x_prime-0.3)+y_prime*y_prime))

b = LinearForm(fes)
b += A * f * v * dx
b.Assemble()


gfu = GridFunction(fes)
gfu.Set(A, BND)
gfu.vec.data = a.mat.Inverse() * b.vec
Draw(gfu);


('inner', 'pmlregion')


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…